## Importing librairies

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import arcpy
import time
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.utils import Sequence
from skimage.transform import resize

arcpy.env.parallelProcessingFactor = "50%"
arcpy.env.pyramid = "NONE"
arcpy.env.rasterStatistics = 'NONE'
arcpy.SetLogMetadata(False)
arcpy.SetLogHistory(False)


In [2]:
# Verification that GPU is active.
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


Defining global variables

In [3]:
# ArcGIS variables
path = "H:/Python/arcpy_tensorflow/"
path_raster = "R:/SIGR/DonneesCommunes/FONDS/ORTHOPHOTO/ortho2021.gdb/orthophoto2021"
raster = arcpy.Raster(path_raster)
img_size = 250 # Size of the images you want to crop
resolution = 0.15 # Resolution of your raster in meters

# Model variables
width, height = 250, 250 # size of the images in the dataset
batch_size = 8
epochs = 10 # Number of epoch for training
name_model = 'Raster-Chantier-inator.h5' # The name of the model, it will save the weights of the model as a .h5 file if you want to use the inference later. 

In [4]:
def processing(array):
    array = array.T
    array = np.rot90(array,k=-1)
    array = np.flip(array,axis=1)
    array = resize(array,(width,height)) # resizing for the model
    return array

def load_image(Coordx,Coordy,img_size=img_size,raster=raster):
    array = arcpy.RasterToNumPyArray(raster,arcpy.Point(Coordx,Coordy),img_size,img_size)
    array = processing(array)
    return array

Creating a data generator using arcpy

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, data_file, batch_size):
        self.data_file = data_file
        self.batch_size = batch_size
        self.num_samples = self.get_num_samples()
        self.num_batches = int(np.ceil(self.num_samples / self.batch_size))

    def get_num_samples(self):
        # Implement your logic to determine the total number of samples
        # in your dataset file and return that value.
        # For example, if your data file is a CSV file, you can use pandas
        # to read the file and count the number of rows.
        # return num_samples
        pass

    def __len__(self):
        return self.num_batches

    def __getitem__(self, index):
        start_index = index * self.batch_size
        end_index = min(start_index + self.batch_size, self.num_samples)

        batch_x, batch_y = self.load_data_chunk(start_index, end_index)

        # Preprocess your data if needed

        return batch_x, batch_y

    def load_data_chunk(self, start_index, end_index):
        # Implement your logic to load a chunk of data from the file
        # based on the given start and end indexes.
        # For example, if your data file is a CSV file, you can use pandas
        # to read the specific rows within the range and extract the
        # features and labels.
        # return batch_x, batch_y
        pass
# Example usage
data_file = 'path/to/your/data/file.csv'  # Replace with your data file path
train_data_generator = DataGenerator(data_file, batch_size)

In [ ]:
# class_names = train_ds.class_names
# print(class_names)

Defining useful functions

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

def time_it(func):
    """
    Usage : Put @time_it before your function to calculate execution time.
    """
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Execution time of {func.__name__}: {end - start:.5f} seconds")
        return result
    return wrapper

Plot of some images to check that everything is going smoothly

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(min(9,batch_size)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds: # Printing the shape of the dataset
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model definition

Here is a list of available model to use as a backbone : https://keras.io/api/applications/

In [ ]:
num_classes = len(class_names)
# Backbone of the model
base_model = tf.keras.applications.EfficientNetB3(input_shape=(height, width,3), include_top=False ,weights='imagenet')
base_model.summary()

In [11]:
last = base_model.output # dernière couche
x = tf.keras.layers.GlobalAveragePooling2D()(last)
# Fully connected layer
x = tf.keras.layers.Dense(4096, activation='relu')(x)
# Logistic layer (number of classes)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model1 = tf.keras.Model(base_model.input, outputs=outputs)

In [12]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
es = EarlyStopping(monitor='val_accuracy', mode='auto', verbose=0,patience=30)
mc = ModelCheckpoint(name_model, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional ResNet152V2 layers
for layer in base_model.layers:
    layer.trainable = False

model1.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# Only train top layers
with tf.device('/device:GPU:0'): # ensure that training is done by GPU just in case
    history = model1.fit(train_ds, validation_data=val_ds,batch_size=batch_size, epochs=epochs,verbose=1, callbacks = [es,mc])

In [ ]:
plot_history(history=history)

In [ ]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)
print(len(model1.layers))

In [49]:
for layer in model1.layers:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
model1.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history11 = model1.fit(train_ds, validation_data=val_ds,batch_size=batch_size, epochs=epochs,verbose=1, callbacks = [es,mc])

In [ ]:
plot_history(history11)

In [ ]:
if data_dir_test != None:
    test_ds = tf.keras.utils.image_dataset_from_directory(
            data_dir_test,
            seed=123,
            image_size=(height, width),
            batch_size=batch_size)
    model1.evaluate(test_ds)